In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [2]:
url_base='https://www.cgmh.org.tw/tw/Systems/'
url_work_table='RecruitInfo/3'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
pages=soup.find_all('ul' ,class_="layout__pagination ul-reset")[0]
def get_pages(pages):
  counter=0
  for i, item in enumerate(pages):
      if item.find('a') :
        s = item.find('a')
        counter += 1
        try:
          #print(s.get("href") ,s.string)
          if s.get("href") == "javascript:void(0)":
            counter -= 1
        except:
          counter -= 1
          #print(item,'error')
      
  return counter
get_pages(pages)


3

In [41]:
def get_work_dead_line( soup):  
  work_detail_web = soup.find('article').get_text().replace("\xa0","").replace('\n',"").replace("\u3000","")
  if '額滿為止' in work_detail_web:
    return '額滿為止'
  elif '自即日起' in work_detail_web:
    return '自即日起'
  elif '即日起至' in work_detail_web:
    dead_line = re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]
    return dead_line
  elif '即日起收件至' in work_detail_web:
    dead_line = work_detail_web.rsplit("即日起收件至")[1].split("止")[0].replace("：","")
    return dead_line      
  else:
    try:
      dead_line = work_detail_web.rsplit("報名期限")[1].split("截止")[0].replace("：","")
    except:
      return re.findall("\d+年\d+月\d+日",work_detail_web)[0]
    return dead_line

In [5]:
def get_work_page(page):
    g=requests.get(url+'?page='+str(page))
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [6]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup    

In [7]:
def get_work_table(soup,tables,work_table):
    for i, item in enumerate(tables):
        if item.find('a'): #過濾掉被刪除的文章
            s = item.find('a')
            url_base_website = urlparse(url)
            work_detail_link = url_base_website._replace(path=urlparse(s.get('href')).path).geturl()
            title = item.find_all('div')[1].string
            work_page_soup = get_work_detail(work_detail_link)
            dead_line = get_work_dead_line( work_page_soup )
            print('#{}召聘職稱: {} 期限: {}\n 連結：{}'.format(i+1, title, dead_line, work_detail_link ))
            work_table.append([i-2, title, dead_line, work_detail_link ])

In [42]:
work_table=[]
for _page in range(1,get_pages(pages)+1):
    soup_=get_work_page(_page)
    tables = soup_.find('div',class_="bg-grey pd100").find_all('ul')[-2].find_all('li')
    get_work_table(soup_,tables,work_table)

#1召聘職稱: 桃園長庚醫院復健科招募語言治療師乙名，請查照。 期限: 2022年06月26日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205230024
#2召聘職稱: 台北長庚醫院招募眼科門診跟診護理師，請查照。 期限: 2022年6月8日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205230039
#3召聘職稱: 林口長庚醫院腎臟科招募醫檢師，請查照。 期限: 2022年06月05日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205200002
#4召聘職稱: 林口長庚實驗動物中心招募專任學士級研究助理 期限: 2022年6月30日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205130038
#5召聘職稱: 基隆長庚醫院招募定期契約臨床心理師，請 查照。 期限: 2022年6月1日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205170029
#6召聘職稱: 高雄市立鳳山醫院招募定期契約個案協調師，請 查照。 期限: 2022年5月26日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205190016
#7召聘職稱: 基隆長庚醫院檢驗醫學科招募定期契約醫事檢驗師 期限: 2022年6月5日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205180020
#8召聘職稱: 高雄長庚醫院放射診斷科招募定契照護師，請 查照。 期限: 2022年05月27日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202205170021
#9召聘職稱: 新北市立土城醫院藥劑科招募藥師3名，請查照。 期限: 2022年06月05日
 連結：https://www.cgmh.org.tw/tw

In [43]:

work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

    no                                召聘職稱                期限  \
0   -2             桃園長庚醫院復健科招募語言治療師乙名，請查照。       2022年06月26日   
1   -1              台北長庚醫院招募眼科門診跟診護理師，請查照。         2022年6月8日   
2    0                 林口長庚醫院腎臟科招募醫檢師，請查照。       2022年06月05日   
3    1               林口長庚實驗動物中心招募專任學士級研究助理        2022年6月30日   
4    2             基隆長庚醫院招募定期契約臨床心理師，請 查照。         2022年6月1日   
5    3           高雄市立鳳山醫院招募定期契約個案協調師，請 查照。        2022年5月26日   
6    4              基隆長庚醫院檢驗醫學科招募定期契約醫事檢驗師         2022年6月5日   
7    5            高雄長庚醫院放射診斷科招募定契照護師，請 查照。       2022年05月27日   
8    6              新北市立土城醫院藥劑科招募藥師3名，請查照。       2022年06月05日   
9    7                       台北長庚中藥局招募一般藥師       2022年05月29日   
10  -2      林口/桃園長庚醫院檢驗醫學部招募部分工時抽血技術員，請查照。       2022年05月29日   
11  -1             嘉義長庚醫院招募血液透析治療員人員，請 查照。        111年06月05日   
12   0         高雄長庚紀念醫院 粒線體醫學研究中心 誠徵博士後研究員              自即日起   
13   1         新北市立土城醫院血液透析中心招募透析治療員，請 查照。       2022年06月05日   
14   2         林口長庚醫院招募胃腸肝膽科系檢查室技術員壹名，請 

In [ ]:
for f in range(1,get_pages(pages)+1):
    print(f)

1
2
3


In [38]:
re.findall("\d+年\d+月\d+日"," 報名期限：即日起至 2022年 6 月 30 日 截止".rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]

'2022年6月30日'